---

# Lib

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from preprocessing.clean import vn_text_clean
from preprocessing.tokenize import vn_word_tokenize
from preprocessing.remove_stopwords import remove_stopwords

[nltk_data] Downloading package punkt to C:\Users\DANG
[nltk_data]     CUONG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

# Read file

In [2]:
df = pd.read_csv("../data/raw/train.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [3]:
df.head()

,brand,model,version,year,segment,comment,label
0,hyundai,creta,n line,2025,suv,"Mẫu này đẹp và hiện đại hơn, mặt đồng hồ trung...",{EXTERIOR#Positive};{INTERIOR#Negative};
1,hyundai,creta,n line,2025,suv,Creta vẫn nên phải ở tầm giá Xforce hoặc thấp ...,{COST#Neutral};{EXTERIOR#Neutral};
2,hyundai,creta,n line,2025,suv,"Giá như cx5 dl, mặc dù con này rộng hơn và k s...",{COST#Positive};
3,hyundai,creta,n line,2025,suv,Giá cao quá đợi chương trình khuyến mại thôi,{COST#Negative};
4,hyundai,creta,n line,2025,suv,Creta là một trong những con xe mượt mà nhất c...,{EXTERIOR#Positive};{INTERIOR#Positive};{PERFO...


In [4]:
df = df[["comment", "label"]].copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  908 non-null    object
 1   label    908 non-null    object
dtypes: object(2)
memory usage: 14.3+ KB


---

# Clean text

In [6]:
df["comment"] = df["comment"].apply(lambda x: vn_text_clean(x))

In [7]:
df.head()

,comment,label
0,mẫu này đẹp và hiện đại hơn mặt đồng hồ trung ...,{EXTERIOR#Positive};{INTERIOR#Negative};
1,creta vẫn nên phải ở tầm giá xforce hoặc thấp ...,{COST#Neutral};{EXTERIOR#Neutral};
2,giá như cx5 dl mặc dù con này rộng hơn và khôn...,{COST#Positive};
3,giá cao quá đợi chương trình khuyến mại thôi,{COST#Negative};
4,creta là một trong những con xe mượt mà nhất c...,{EXTERIOR#Positive};{INTERIOR#Positive};{PERFO...


In [8]:
df.iloc[0, 0]

'mẫu này đẹp và hiện đại hơn mặt đồng hồ trung tâm đã làm âm vào mặt taplo chứ không như cái ipad dựng đứng tạo ra khoảng trống phía sau lái xe có để gì ở sau sẽ không nhìn thấy'

---

# Tokenize

In [9]:
df["comment"] = df["comment"].apply(vn_word_tokenize, method="underthesea")

In [10]:
df.head()

,comment,label
0,mẫu này đẹp và hiện_đại hơn mặt đồng_hồ trung_...,{EXTERIOR#Positive};{INTERIOR#Negative};
1,creta vẫn nên phải ở tầm giá xforce hoặc thấp ...,{COST#Neutral};{EXTERIOR#Neutral};
2,giá như cx5 dl mặc_dù con này rộng hơn và khôn...,{COST#Positive};
3,giá cao quá đợi chương_trình khuyến_mại thôi,{COST#Negative};
4,creta là một trong những con xe mượt_mà nhất c...,{EXTERIOR#Positive};{INTERIOR#Positive};{PERFO...


In [11]:
df.iloc[0, 0]

'mẫu này đẹp và hiện_đại hơn mặt đồng_hồ trung_tâm đã làm âm vào mặt taplo chứ không như cái ipad dựng_đứng tạo ra khoảng trống phía sau lái_xe có để gì ở sau sẽ không nhìn thấy'

In [12]:
# Backup comment for remove stopwords
df["comment_backup"] = df["comment"].copy()

---

# Remove stopwords

In [13]:
# Remove stopwords with fallback if result becomes empty/NaN
post = df["comment"].apply(remove_stopwords)

# Normalize empties to NaN to enable fallback
post = post.replace("", np.nan)

In [14]:
post.isnull().sum()

np.int64(2)

In [15]:
# Fallback to backup text where stopword removal produced NaN
fallback = post.fillna(df["comment_backup"]) 

df["comment"] = fallback

In [16]:
df.head()

,comment,label,comment_backup
0,mẫu đẹp hiện_đại mặt đồng_hồ trung_tâm âm mặt ...,{EXTERIOR#Positive};{INTERIOR#Negative};,mẫu này đẹp và hiện_đại hơn mặt đồng_hồ trung_...
1,creta tầm giá xforce chút phiên_bản rẻ thiết_k...,{COST#Neutral};{EXTERIOR#Neutral};,creta vẫn nên phải ở tầm giá xforce hoặc thấp ...
2,giá cx5 dl mặc_dù rộng say cx5,{COST#Positive};,giá như cx5 dl mặc_dù con này rộng hơn và khôn...
3,giá đợi chương_trình khuyến_mại,{COST#Negative};,giá cao quá đợi chương_trình khuyến_mại thôi
4,creta xe mượt_mà hyundai phân_khúc khung gầm t...,{EXTERIOR#Positive};{INTERIOR#Positive};{PERFO...,creta là một trong những con xe mượt_mà nhất c...


In [17]:
df.iloc[0, 0]

'mẫu đẹp hiện_đại mặt đồng_hồ trung_tâm âm mặt taplo ipad dựng_đứng trống lái_xe'

---

# Save file

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   comment         908 non-null    object
 1   label           908 non-null    object
 2   comment_backup  908 non-null    object
dtypes: object(3)
memory usage: 21.4+ KB


In [19]:
df.drop(columns="comment_backup", axis=1, inplace=True)

In [20]:
df.to_csv("../data/processed/train_preprocessed.csv", index=False)